In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
! pip install split-folders

import split_folders

Using TensorFlow backend.


In [0]:
# Split the data into training and validation
training_data_folder = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training"
split_folders.ratio(training_data_folder, output = "train_val_test", seed=1337, ratio = (.6,.3,.1))

Copying files: 4476 files [32:58,  2.28 files/s]


In [0]:
# Data Initialization
import os

# ! unzip "train_val.zip"
train_data_dir = "train_val_test/train"
val_data_dir = "train_val_test/val"
num_train = 0
num_val = 1344

# for i in os.listdir(train_data_dir):
#   num_train += len(os.listdir(os.path.join(train_data_dir, i)))


img_width, img_height = 224, 224
# Check if the images are RGB and change the channels likewise
if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)


In [0]:

# CNN model
model = Sequential()
model.add(Conv2D(32,(2,2), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,(2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [0]:
batch_size = 64
epochs = 50

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2)
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size =(img_width, img_height), batch_size = batch_size, class_mode = 'categorical')
validation_generator = train_datagen.flow_from_directory( val_data_dir, target_size =(img_width, img_height), batch_size = batch_size, class_mode ='categorical') 

model.fit_generator(train_generator, steps_per_epoch = num_train // batch_size, epochs = epochs, validation_data = validation_generator, validation_steps = num_val// batch_size)

model.save_weights("v1_1.h5")

In [0]:
! zip -r "/content/train_val_test.zip" "/content/train_val_test"
test_set_dir = ""

In [0]:
#  ! unzip "/content/drive/My Drive/NN-ProjectC/Project_C1/train_val_test.zip"
import numpy
import sklearn.metrics as metrics
img_width, img_height = 224, 224
batch_size = 64
epochs = 50

test_set_dir = "/content/content/train_val_test/test"

num_test = 0
for i in os.listdir(test_set_dir):
  num_test += len(os.listdir(os.path.join(test_set_dir, i)))

print ("Number of images in test set: ", num_test)

model.load_weights("/content/drive/My Drive/NN-ProjectC/Project_C1/Models/v1_1.h5")
  
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_set_dir, target_size =(img_width, img_height), 
                                                  batch_size = batch_size, class_mode ='categorical') 
# scoreSeg = model.evaluate_generator(test_generator, num_test // batch_size)
# scoreSeg[1]

test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict_generator(test_generator, steps = test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)
true_classes = test_generator.classes
print (len(predicted_classes))
print (len(true_classes))
class_labels = list(test_generator.class_indices.keys()) 
report = metrics.confusion_matrix(true_classes, predicted_classes, labels=[0,1,2,3,4])
print(report)

Number of images in test set:  450
Found 450 images belonging to 5 classes.
450
450
[[95 42 31 32 19]
 [38 12 10 10  7]
 [34  6  7  5  5]
 [30  7  8 11  3]
 [17 11  4  3  3]]
